In [1]:
import numpy as np
import pandas as pd
import gc
import time
import warnings
import lightgbm as lgb;
warnings.simplefilter(action = 'ignore', category = FutureWarning)
from scipy import stats;
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold
from bayes_opt import BayesianOptimization;
from lightgbm import LGBMClassifier
from scipy.stats import ranksums

In [ ]:
dt = pd.read_csv("../input/ds9.csv") # loading our pooled features, about 2k features


CATEGORICAL_COLUMNS = ['CODE_GENDER',
                       'EMERGENCYSTATE_MODE',
                       'FLAG_CONT_MOBILE',
                       'FLAG_DOCUMENT_3',
                       'FLAG_DOCUMENT_4',
                       'FLAG_DOCUMENT_5',
                       'FLAG_DOCUMENT_6',
                       'FLAG_DOCUMENT_7',
                       'FLAG_DOCUMENT_8',
                       'FLAG_DOCUMENT_9',
                       'FLAG_DOCUMENT_11',
                       'FLAG_DOCUMENT_18',
                       'FLAG_EMAIL',
                       'FLAG_EMP_PHONE',
                       'FLAG_MOBIL',
                       'FLAG_OWN_CAR',
                       'FLAG_OWN_REALTY',
                       'FLAG_PHONE',
                       'FLAG_WORK_PHONE',
                       'FONDKAPREMONT_MODE',
                       'HOUR_APPR_PROCESS_START',
                       'HOUSETYPE_MODE',
                       'LIVE_CITY_NOT_WORK_CITY',
                       'LIVE_REGION_NOT_WORK_REGION',
                       'NAME_CONTRACT_TYPE',
                       'NAME_TYPE_SUITE',
                       'NAME_INCOME_TYPE',
                       'NAME_EDUCATION_TYPE',
                       'NAME_FAMILY_STATUS',
                       'NAME_HOUSING_TYPE',
                       'OCCUPATION_TYPE',
                       'ORGANIZATION_TYPE',
                       'REG_CITY_NOT_LIVE_CITY',
                       'REG_CITY_NOT_WORK_CITY',
                       'REG_REGION_NOT_LIVE_REGION',
                       'REG_REGION_NOT_WORK_REGION',
                       'WALLSMATERIAL_MODE',
                       'WEEKDAY_APPR_PROCESS_START']


train_df = dt.iloc[:307511,]
test_df = dt.iloc[307511:,]

In [ ]:
train_id = train_df.SK_ID_CURR
test_id = test_df.SK_ID_CURR

train_df.drop(["TARGET.x","SK_ID_CURR"],axis=1,inplace = True)
test_df.drop(["TARGET.x","SK_ID_CURR"],axis=1,inplace = True)

fold_df = pd.read_csv('../input/folds.csv') # importing folds (actually generated by sklearn KFold class with shuffle=True and seed=1001 then dumped to a csv so all team members use the same fold indices for creationg new oofs)
Y = fold_df.Y

## Actually it is not for parameter optimiztion, it is for creation of new and different oofs; But how ? 

well; every now and then we dump a good candidate solution as an oof...
You can change it to work with Catboost or XGBoost easily

In [ ]:
def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    params = {'application':'binary','num_iterations':4000, 'learning_rate':0.05, 'early_stopping_round':100, 'metric':'auc'}
    params["num_leaves"] = round(num_leaves)
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
    params['max_depth'] = round(max_depth)
    params['lambda_l1'] = max(lambda_l1, 0)
    params['lambda_l2'] = max(lambda_l2, 0)
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    cv_result = lgb_cv(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight)
    return cv_result;

def lgb_cv(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    Y =  np.array(fold_df.Y);
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    for i in range(1,6):
        for j in [0]:
            ft_to_use = train_df.columns.values;
            CATEGORICAL_COLUMNS_X = [f for f in ft_to_use if f in CATEGORICAL_COLUMNS]
            dtest = lgb.Dataset(data=test_df, 
                                categorical_feature = CATEGORICAL_COLUMNS_X,
                                free_raw_data=False, silent=True)
            ix_valid = fold_df[fold_df.FoldID==i].index;
            ix_train = fold_df[fold_df.FoldID!=i].index;
            x_train = train_df.iloc[ix_train];
            x_train = x_train;
            v_train = train_df.iloc[ix_valid];
            v_train = v_train;
    
            y_train = Y[ix_train];
            y_valid = Y[ix_valid];
            dtrain = lgb.Dataset(data=x_train, 
                                 label=y_train, 
                                 categorical_feature = CATEGORICAL_COLUMNS_X,
                                 free_raw_data=False, silent=True)
            dvalid = lgb.Dataset(data=v_train, 
                                 label=y_valid, 
                                 categorical_feature = CATEGORICAL_COLUMNS_X,
                                 free_raw_data=False, silent=True)
        
            #multiple runs
            params = {
                    'objective': 'binary',
                    'boosting_type': 'gbdt',
                    'nthread': 10,
                    'learning_rate': 0.02,  # 02,
                    'num_leaves': int(num_leaves),
                    'colsample_bytree': feature_fraction,
                    'subsample': bagging_fraction,
                    'subsample_freq': 1,
                    'max_depth': int(max_depth),
                    'reg_alpha': lambda_l1,
                    'reg_lambda': lambda_l2,
                    'min_split_gain': min_split_gain,
                    'min_child_samples': int(min_child_weight), # 39.3259775,
                    'seed': 0,
                    'verbose': -1,
                    'metric': 'auc',
            }
            clf = lgb.train(
                params=params,
                train_set=dtrain,
                num_boost_round=100000,
                categorical_feature = CATEGORICAL_COLUMNS_X,
                valid_sets=[dtrain, dvalid],
                early_stopping_rounds=200,
                verbose_eval=500
            );
            oof_preds[ix_valid] += clf.predict(dvalid.data)
            sub_preds += clf.predict(dtest.data);
            sc = roc_auc_score(y_valid, oof_preds[ix_valid]);

    #full test
    oof_preds = oof_preds;
    sub_preds = sub_preds / 5;   
    
    sc = roc_auc_score(Y, oof_preds);
    submission_file_name = "test_" + str.replace(str(sc),'.','_') + ".csv";
    oof_file_name = "oof_" + str.replace(str(sc),'.','_') + ".csv";
    sub_df = pd.DataFrame(columns=["SK_ID_CURR"],data=test_id);
    sub_df['TARGET'] = sub_preds
    sub_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    oof_df = pd.DataFrame(columns=["SK_ID_CURR"],data=train_id);
    oof_df['TARGET'] = oof_preds
    oof_df[['SK_ID_CURR', 'TARGET']].to_csv(oof_file_name, index= False)
    print('Full AUC score %.6f' % roc_auc_score(Y, oof_preds))            
    x_train  = None;
    v_train = None;
    dtrain  = None;
    dvalid = None;
    dtest = None;
    gc.collect();
    return sc;


gbBO = BayesianOptimization(lgb_eval, {'num_leaves': (18, 40),
                                        'feature_fraction': (0.02, 0.2),
                                        'bagging_fraction': (0.8, 1),
                                        'max_depth': (-1,8),
                                        'lambda_l1': (0, 100),
                                        'lambda_l2': (0, 100),
                                        'min_split_gain': (0.0, 0.5),
                                        'min_child_weight': (10, 100)}, 
                                        random_state=0)
            
            
gbBO.maximize(init_points=15, n_iter=100);

